In [7]:
import os

BASE_READ_PATH = None
EMBEDDINGS_WRITE_PATH = None

try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_name = 'CSCI544-Project'
    folder_path = f'/content/drive/MyDrive/{folder_name}'
    os.chdir(folder_path)
    print("Current working directory is: " + os.getcwd())
    BASE_READ_PATH = "./data/"
    EMBEDDINGS_WRITE_PATH = "./t5p_small_embeddings/"
except:
    BASE_READ_PATH = "../data/"
    EMBEDDINGS_WRITE_PATH = "../t5p_small_embeddings/"
finally:
    print("Current working directory is: " + os.getcwd())

Current working directory is: /Users/swapnildhanwal/Documents/USC/CSCI 544/Project/csci-544-project/src
['embeddings.ipynb', 'training.ipynb', 'embeddings.py', 'main.py', 'data.py']


In [8]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import json
import os

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch

In [10]:
CHECKPOINT = "Salesforce/codet5p-110m-embedding"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TOKENIZER = AutoTokenizer.from_pretrained(CHECKPOINT, trust_remote_code=True)
MODEL = AutoModel.from_pretrained(CHECKPOINT, trust_remote_code=True).to(DEVICE)

In [11]:
def get_code_embedding(code: str):
    inputs = TOKENIZER.encode(code, return_tensors="pt").to(DEVICE)
    embedding = MODEL(inputs)[0]
    embedding_np_array = embedding.reshape(-1, embedding.shape[0]).cpu().detach().numpy()
    del embedding
    return embedding_np_array


def get_and_save_embeddings(file_name: str, _type: str):
    read_path = "%s%s/%s" % (BASE_READ_PATH, _type, file_name)
    write_path = "%s%s/%s" % (EMBEDDINGS_WRITE_PATH, _type, file_name)
    num_lines = get_file_length(read_path)
    with open(read_path, "r") as f_read:
        i = 0
        with open(write_path, "w") as f_write:
            with tqdm(total=num_lines) as bar:
                for line in f_read.readlines():
                    if i % 30 == 0:
                        torch.cuda.empty_cache()
                    try:
                        data = json.loads(line)
                        code = data["func_before"]
                        embeddings = get_code_embedding(code)
                        label = int(data["vul"])
                        result = {"embeddings": embeddings.tolist(), "label": label}
                        f_write.write(json.dumps(result) + "\n")
                    except Exception as e:
                        print(e)
                    finally:
                        i += 1
                        bar.update()


def get_file_length(file_path):
    count = 0
    with open(file_path, "r") as f:
        line = f.readline()
        while True:
            if not line:
                break
            count += 1
            line = f.readline()
    return count


def main(_type: str):
    for file_name in os.listdir("%s%s/" % (BASE_READ_PATH, _type)):
        print("Generating embeddings for: " + file_name)
        get_and_save_embeddings(file_name, _type)

In [12]:
_type = "train"
main(_type)

Generating embeddings for: train_0.jsonl


FileNotFoundError: [Errno 2] No such file or directory: '../t5p_small_embeddings/train/train_0.jsonl'